In [84]:
import pandas as pd
import plotly.express as px
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np

In [46]:
dailyobs_df = pd.read_csv("daily_obs.csv")
dailyobs_df.head()

,id,obs_date_time,observer,obs_location,sky_cover,precip_type,precip_rate,air_temp_min,air_temp_max,air_temp_current,...,rain,accumulated_precip,blowing_snow,blowing_snow_dir,frz_lvl_min,frz_lvl_max,frz_lvl_cur,sno_stl,inversion,sh_nsf_obs
0,351,11/10/2015 6:00,mike,Mt Roberts Tram Wx,OVC,RS,S-1,33.0,34.8,33.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,352,11/11/2015 6:00,mike,Mt Roberts Tram Wx,OVC,SN,S-1,29.9,33.5,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,353,11/12/2015 6:00,mike,Mt Roberts Tram Wx,OVC,SN,S2,29.6,32.3,31.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,354,11/13/2015 6:00,mike,Mt Roberts Tram Wx,OVC,SN,S2,31.6,32.4,31.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,355,11/13/2015 6:00,mike,Speel Arm Balcony Wx,OVC,SN,S-1,30.6,32.5,31.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Create new DataFrame with only important columns from the daily_obs DataFrame
daily_obs_clean = dailyobs_df[['obs_date_time', 'obs_location', 'sky_cover', 'precip_type', 
                            'air_temp_min', 'air_temp_max', 'air_temp_current',
                            'snow_height', 'new_snow_height', 'wind_direction', 'wind_speed',
                            'wind_gust', 'hazard']]
daily_obs_clean.head()

,obs_date_time,obs_location,sky_cover,precip_type,air_temp_min,air_temp_max,air_temp_current,snow_height,new_snow_height,wind_direction,wind_speed,wind_gust,hazard
0,11/10/2015 6:00,Mt Roberts Tram Wx,OVC,RS,33.0,34.8,33.5,2.8,NaN,SW,5.0,10.0,0.0
1,11/11/2015 6:00,Mt Roberts Tram Wx,OVC,SN,29.9,33.5,30.0,4.3,NaN,SE,11.0,14.0,0.0
2,11/12/2015 6:00,Mt Roberts Tram Wx,OVC,SN,29.6,32.3,31.9,12.6,7.0,SE,27.0,42.0,0.0
3,11/13/2015 6:00,Mt Roberts Tram Wx,OVC,SN,31.6,32.4,31.7,14.2,5.0,SE,26.0,29.0,0.0
4,11/13/2015 6:00,Speel Arm Balcony Wx,OVC,SN,30.6,32.5,31.4,19.0,5.0,SSW,7.1,23.3,0.0


In [48]:
daily_obs_clean.isnull().sum()

obs_date_time          0
obs_location           0
sky_cover            350
precip_type         1509
air_temp_min        1165
air_temp_max        1163
air_temp_current    1519
snow_height         1700
new_snow_height     1664
wind_direction       693
wind_speed           732
wind_gust            644
hazard               419
dtype: int64

In [49]:
# Drop Null value
daily_obs_clean = daily_obs_clean.dropna()

In [50]:
# Check to make sure Null values were dropped
daily_obs_clean.isnull().sum()

obs_date_time       0
obs_location        0
sky_cover           0
precip_type         0
air_temp_min        0
air_temp_max        0
air_temp_current    0
snow_height         0
new_snow_height     0
wind_direction      0
wind_speed          0
wind_gust           0
hazard              0
dtype: int64

In [51]:
daily_obs_clean.shape

(3512, 13)

In [52]:
#Drop observation date
daily_obs_clean = daily_obs_clean.drop('obs_date_time',axis=1)

In [53]:
#Encode wind direction 
wind_direction_counts = daily_obs_clean.wind_direction.value_counts()
wind_direction_counts

# Determine which values to replace
replace_wind_direction = list(wind_direction_counts[wind_direction_counts < 150].index)

# Replace in DataFrame
for direction in replace_wind_direction:
    daily_obs_clean.wind_direction = daily_obs_clean.wind_direction.replace(direction,"Other")

In [54]:
#Encode location
obs_location_counts = daily_obs_clean.obs_location.value_counts()
obs_location_counts

# Determine which values to replace
replace_obs_location = list(obs_location_counts[obs_location_counts < 200].index)

# Replace in DataFrame
for location in replace_obs_location:
    daily_obs_clean.obs_location = daily_obs_clean.obs_location.replace(location,"Other")

In [55]:
#encode categorical columns
X = pd.get_dummies(daily_obs_clean, columns=["obs_location", "wind_direction", "sky_cover", "precip_type"])
X

,air_temp_min,air_temp_max,air_temp_current,snow_height,new_snow_height,wind_speed,wind_gust,hazard,obs_location_4-4 Diverter,obs_location_Mt Roberts Tram,...,sky_cover_FEW,sky_cover_OVC,sky_cover_SCT,sky_cover_X,precip_type_GR,precip_type_NO,precip_type_RA,precip_type_RS,precip_type_SN,precip_type_ZR
2,29.6,32.3,31.9,12.6,7.0,27.0,42.0,0.0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,31.6,32.4,31.7,14.2,5.0,26.0,29.0,0.0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,30.6,32.5,31.4,19.0,5.0,7.1,23.3,0.0,0,0,...,0,1,0,0,0,0,0,0,1,0
7,31.6,32.4,31.8,22.4,7.0,10.0,10.0,0.0,0,0,...,0,1,0,0,0,0,0,0,1,0
8,31.2,33.4,31.2,0.0,0.0,1.1,25.1,0.0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6038,-14.3,-10.7,-10.7,99.2,1.0,0.0,0.0,1.0,0,0,...,0,0,0,1,0,0,0,0,1,0
6039,-6.6,-3.2,-4.2,142.3,7.2,3.9,9.8,1.0,0,0,...,0,0,0,0,0,1,0,0,0,0
6040,-6.5,-3.8,-6.2,14.0,5.0,0.8,5.3,1.0,0,0,...,0,0,0,0,0,1,0,0,0,0
6041,-8.3,-3.6,-5.9,85.0,9.0,1.9,6.3,1.0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [56]:
X.dtypes

air_temp_min                              float64
air_temp_max                              float64
air_temp_current                          float64
snow_height                               float64
new_snow_height                           float64
wind_speed                                float64
wind_gust                                 float64
hazard                                    float64
obs_location_4-4 Diverter                   uint8
obs_location_Mt Roberts Tram                uint8
obs_location_Mt Roberts Tram Combo Obs      uint8
obs_location_Other                          uint8
obs_location_SS Creek DOT                   uint8
obs_location_Snettisham Combo Obs           uint8
obs_location_Snettisham Dorm                uint8
obs_location_Speel Arm Balcony              uint8
obs_location_Thane Road Combo Obs           uint8
wind_direction_E                            uint8
wind_direction_ENE                          uint8
wind_direction_ESE                          uint8


In [57]:
#Scale data
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[ 5.69455775  4.99884527  5.84278865 ... -0.26534009  2.10630779
  -0.07567943]
 [ 6.06460106  5.01587083  5.80591447 ... -0.26534009  2.10630779
  -0.07567943]
 [ 5.87957941  5.03289639  5.7506032  ... -0.26534009  2.10630779
  -0.07567943]
 ...
 [-0.98472403 -1.14738341 -1.18174298 ... -0.26534009 -0.47476442
  -0.07567943]
 [-1.31776301 -1.11333228 -1.12643171 ... -0.26534009 -0.47476442
  -0.07567943]
 [-1.78031715 -1.79435485 -1.5504848  ... -0.26534009 -0.47476442
  -0.07567943]]


In [58]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=5)
daily_obs_pca = pca.fit_transform(X_scaled)
daily_obs_pca

array([[ 8.71701421,  5.20863742,  5.58724185,  4.0816855 ,  2.78705644],
       [ 8.8038006 ,  4.43832956,  4.43238156,  3.86549541,  2.7398721 ],
       [ 8.33709641,  2.25469891,  0.19793433,  4.49592058,  1.51713056],
       ...,
       [-1.89117398, -1.61944832,  0.62683909, -0.61111426,  0.35550558],
       [-2.52379463, -1.02544104,  0.52905552,  0.90965329,  0.39691206],
       [-3.65226174, -0.93251931,  0.5617074 , -0.54225032, -1.59806149]])

In [59]:
# Create a DataFrame with the three principal components.
daily_obs_df = pd.DataFrame(
    data=daily_obs_pca, columns=["PC 1", "PC 2", "PC 3", "PC 4", "PC 5"], index = X.index
)
daily_obs_df.head()

,PC 1,PC 2,PC 3,PC 4,PC 5
2,8.717014,5.208637,5.587242,4.081685,2.787056
3,8.803801,4.438330,4.432382,3.865495,2.739872
4,8.337096,2.254699,0.197934,4.495921,1.517131
7,8.200834,1.869639,-0.177174,4.117701,1.033289
8,9.668490,0.748079,0.040099,3.627646,1.327895


In [60]:
inertia = []
k = list(range(1, 11))

In [61]:
# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(daily_obs_df)
    inertia.append(km.inertia_)

In [62]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [63]:
def get_clusters(k, data):
    # Create a copy of the DataFrame
    data = daily_obs_df.copy()

    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Fit the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["class"] = model.labels_

    return data

In [64]:
#Five clusters
five_clusters = get_clusters(5, daily_obs_df)
five_clusters.head()

,PC 1,PC 2,PC 3,PC 4,PC 5,class
2,8.717014,5.208637,5.587242,4.081685,2.787056,4
3,8.803801,4.438330,4.432382,3.865495,2.739872,4
4,8.337096,2.254699,0.197934,4.495921,1.517131,0
7,8.200834,1.869639,-0.177174,4.117701,1.033289,0
8,9.668490,0.748079,0.040099,3.627646,1.327895,0


In [65]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca3 = pca.fit_transform(five_clusters)
pca3

array([[ 5.31476717, -1.18349838, 10.83398894],
       [ 5.54881814, -1.46041345,  9.57878734],
       [ 7.53994079, -0.81021743,  4.00722197],
       ...,
       [-1.97849389, -1.06589891, -0.8311852 ],
       [-2.60709134, -0.49914086, -0.54906276],
       [-3.25093324,  0.24008895, -1.29105603]])

In [66]:
# Create a DataFrame with the three principal components.
pca3_df = pd.DataFrame(
    data=pca3, columns=["PC 1", "PC 2", "PC 3"], index = X.index
)
pca3_df.head()

,PC 1,PC 2,PC 3
2,5.314767,-1.183498,10.833989
3,5.548818,-1.460413,9.578787
4,7.539941,-0.810217,4.007222
7,7.535896,-0.891069,3.375251
8,8.685747,-2.332691,3.358737


In [89]:
pca_out = PCA().fit(pca3_df)

# get the component variance
# Proportion of Variance (from PC1 to PC6)
pca_out.explained_variance_ratio_

array([0.43287209, 0.29550846, 0.27161945])

In [85]:
# Cumulative proportion of variance (from PC1 to PC6)   
np.cumsum(pca_out.explained_variance_ratio_)


array([0.43287209, 0.72838055, 1.        ])

In [91]:
# component loadings or weights (correlation coefficient between original variables and the component) 
# component loadings represents the elements of the eigenvector
# the squared loadings within the PCs always sums to 1
loadings = pca_out.components_
num_pc = pca_out.n_features_
pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
loadings_df = pd.DataFrame.from_dict(dict(zip(pc_list, loadings)))
loadings_df['variable'] = X.columns.values
loadings_df = loadings_df.set_index('variable')
loadings_df

ValueError: Length of values (39) does not match length of index (3)

In [80]:
inertia = []
k = list(range(1, 11))

# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca3_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [67]:
#Five clusters
five2_clusters = get_clusters(5, pca3_df)
five2_clusters.head()

,PC 1,PC 2,PC 3,PC 4,PC 5,class
2,8.717014,5.208637,5.587242,4.081685,2.787056,4
3,8.803801,4.438330,4.432382,3.865495,2.739872,4
4,8.337096,2.254699,0.197934,4.495921,1.517131,0
7,8.200834,1.869639,-0.177174,4.117701,1.033289,0
8,9.668490,0.748079,0.040099,3.627646,1.327895,0


In [68]:
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([X, pca3_df], axis=1)
clustered_df.head()

,air_temp_min,air_temp_max,air_temp_current,snow_height,new_snow_height,wind_speed,wind_gust,hazard,obs_location_4-4 Diverter,obs_location_Mt Roberts Tram,...,sky_cover_X,precip_type_GR,precip_type_NO,precip_type_RA,precip_type_RS,precip_type_SN,precip_type_ZR,PC 1,PC 2,PC 3
2,29.6,32.3,31.9,12.6,7.0,27.0,42.0,0.0,0,0,...,0,0,0,0,0,1,0,5.314767,-1.183498,10.833989
3,31.6,32.4,31.7,14.2,5.0,26.0,29.0,0.0,0,0,...,0,0,0,0,0,1,0,5.548818,-1.460413,9.578787
4,30.6,32.5,31.4,19.0,5.0,7.1,23.3,0.0,0,0,...,0,0,0,0,0,1,0,7.539941,-0.810217,4.007222
7,31.6,32.4,31.8,22.4,7.0,10.0,10.0,0.0,0,0,...,0,0,0,0,0,1,0,7.535896,-0.891069,3.375251
8,31.2,33.4,31.2,0.0,0.0,1.1,25.1,0.0,0,0,...,0,0,0,1,0,0,0,8.685747,-2.332691,3.358737


In [69]:
clustered_df["class"] = five2_clusters["class"]
clustered_df

,air_temp_min,air_temp_max,air_temp_current,snow_height,new_snow_height,wind_speed,wind_gust,hazard,obs_location_4-4 Diverter,obs_location_Mt Roberts Tram,...,precip_type_GR,precip_type_NO,precip_type_RA,precip_type_RS,precip_type_SN,precip_type_ZR,PC 1,PC 2,PC 3,class
2,29.6,32.3,31.9,12.6,7.0,27.0,42.0,0.0,0,0,...,0,0,0,0,1,0,5.314767,-1.183498,10.833989,4
3,31.6,32.4,31.7,14.2,5.0,26.0,29.0,0.0,0,0,...,0,0,0,0,1,0,5.548818,-1.460413,9.578787,4
4,30.6,32.5,31.4,19.0,5.0,7.1,23.3,0.0,0,0,...,0,0,0,0,1,0,7.539941,-0.810217,4.007222,0
7,31.6,32.4,31.8,22.4,7.0,10.0,10.0,0.0,0,0,...,0,0,0,0,1,0,7.535896,-0.891069,3.375251,0
8,31.2,33.4,31.2,0.0,0.0,1.1,25.1,0.0,0,0,...,0,0,1,0,0,0,8.685747,-2.332691,3.358737,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6038,-14.3,-10.7,-10.7,99.2,1.0,0.0,0.0,1.0,0,0,...,0,0,0,0,1,0,-1.845952,3.839601,-1.771685,1
6039,-6.6,-3.2,-4.2,142.3,7.2,3.9,9.8,1.0,0,0,...,0,1,0,0,0,0,-3.060180,0.377425,1.842956,2
6040,-6.5,-3.8,-6.2,14.0,5.0,0.8,5.3,1.0,0,0,...,0,1,0,0,0,0,-1.978494,-1.065899,-0.831185,2
6041,-8.3,-3.6,-5.9,85.0,9.0,1.9,6.3,1.0,0,0,...,0,1,0,0,0,0,-2.607091,-0.499141,-0.549063,2


In [72]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    hover_data=['air_temp_min'],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [73]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="air_temp_current",
    y="new_snow_height",
    z="wind_speed",
    color="hazard",
    symbol="hazard",
    hover_data=['air_temp_min'],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [79]:
clustered_df.hvplot.bar(x="hazard", y="new_snow_height")

:Bars   [hazard]   (new_snow_height)